In [4]:
import numpy as NP
import tensorflow as TF
import keras.backend as K


Using TensorFlow backend.


In [2]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param 
        session: 
                The TensorFlow session to be frozen.
        keep_var_names: 
                A list of variable names that should not be frozen,
                or None to freeze all the variables in the graph.
        output_names:
                Names of the relevant graph outputs.
        clear_devices:
                Remove the device directives from the graph for better portability.
    @return 
        The frozen graph definition.
    """
    from tensorflow.python.framework.graph_util import convert_variables_to_constants as vars2consts
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in TF.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in TF.global_variables()]
        # Graph -> GraphDef ProtoBuf
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = vars2consts(session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph
    

In [3]:
import sys
sys.path.insert(1, r"D:\HARI\__RL_Dev__\__Main_UTL__")
from yolo_v3 import YOLO

yolo = YOLO()
print('[YOLO] Input tensors:', yolo.yolo_model.inputs)
print('[YOLO] Output tensors:', yolo.yolo_model.outputs)


D:\HARI\__RL_Dev__\__Main_UTL__ --> yolo_v3.py



[YOLO_v3] Initializing










[YOLO_v3] Session is created









[YOLO_v3] Loading weights ...



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



[YOLO_v3] Initialized successfully



[YOLO] Input tensors: [<tf.Tensor 'input_1:0' shape=(?, ?, ?, 3) dtype=float32>]
[YOLO] Output tensors: [<tf.Tensor 'conv2d_59/BiasAdd:0' shape=(?, ?, ?, 27) dtype=float32>, <tf.Tensor 'conv2d_67/BiasAdd:0' shape=(?, ?, ?, 27) dtype=float32>, <tf.Tensor 'conv2d_75/BiasAdd:0' shape=(?, ?, ?, 27) dtype=float32>]


In [4]:
frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in yolo.yolo_model.outputs])
TF.train.write_graph(frozen_graph, "pb_models", "UTL_YOLO.pb", as_text=False)
TF.train.write_graph(frozen_graph, "pb_models", "UTL_YOLO.pbtxt", as_text=True)


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 654 variables.
INFO:tensorflow:Converted 654 variables to const ops.


'pb_models\\UTL_YOLO.pbtxt'

In [7]:
gFile = TF.gfile.FastGFile("./pb_models/UTL_YOLO.pb", 'rb')
graph_def = TF.GraphDef()

# Parses a serialized binary message into the current message.
graph_def.ParseFromString(gFile.read())
gFile.close()

sess = TF.Session()
sess.graph.as_default()

# Import a serialized TensorFlow `GraphDef` protocol buffer
# and place into the current default `Graph`.
TF.import_graph_def(graph_def)


NotFoundError: NewRandomAccessFile failed to Create/Open: pb_models/UTL_YOLO.pb : The system cannot find the path specified.
; No such process

In [2]:
from IPython.display import clear_output, Image, display, HTML


def strip_consts(graph_def, max_const_size=32):

    """Strip large constant values from graph_def."""
    strip_def = TF.GraphDef()

    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)

            if size > max_const_size:
                tensor.tensor_content = bytes("<stripped %d bytes>"%size, encoding='utf-8')
    return strip_def


def show_graph(graph_def, max_const_size=32):

    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()

    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))

    display(HTML(iframe))
    

In [ ]:
show_graph(TF.get_default_graph().as_graph_def())

In [ ]:
input_tensor = sess.graph.get_tensor_by_name('import/input_1:0')
bbox_tensor = sess.graph.get_tensor_by_name('import/conv2d_59/BiasAdd:0')
score_tensor = sess.graph.get_tensor_by_name('import/conv2d_67/BiasAdd:0')
class_tensor = sess.graph.get_tensor_by_name('import/conv2d_75/BiasAdd:0')